In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

st.set_page_config(page_title="Dashboard Inspeção EPI", layout="wide")

st.title("Dashboard de Pendências de Inspeção de EPI")

# === Simulando seus dados (substitua pelo seu pd.read_csv ou outra fonte) ===

# Planilha KIT_EPI simulada
kit_epi = pd.DataFrame({
    'IDTEL_TECNICO': [101, 102, 103, 104, 105],
    'PRODUTO_PRINC_INSPECAO': ['EPI1', 'EPI2', 'EPI1', 'EPI3', 'EPI2']
})

# Planilha INSPECAO simulada
inspecao = pd.DataFrame({
    'IDTEL': [101, 101, 102, 104],
    'DATA_INSPECAO': pd.to_datetime([
        '2024-05-01', '2024-01-01', '2023-12-01', '2023-11-15'
    ])
})

# Planilha MICROSIGA simulada
microsiga = pd.DataFrame({
    'IDTEL': [101, 102, 103, 104, 105],
    'GERENTE_IMEDIATO': ['Gerente A', 'Gerente A', 'Gerente B', 'Gerente B', 'Gerente A'],
    'COORDENADOR_IMEDIATO': ['Coord 1', 'Coord 2', 'Coord 1', 'Coord 3', 'Coord 2']
})

# === Processamento ===

# Última inspeção por técnico
ultima_inspecao = (
    inspecao.groupby('IDTEL')['DATA_INSPECAO']
    .max()
    .reset_index()
)

# Merge dados
df = (
    kit_epi
    .merge(ultima_inspecao, left_on='IDTEL_TECNICO', right_on='IDTEL', how='left')
    .merge(microsiga[['IDTEL', 'GERENTE_IMEDIATO', 'COORDENADOR_IMEDIATO']], left_on='IDTEL_TECNICO', right_on='IDTEL', how='left')
)

# Definindo prazo de 180 dias para pendência
hoje = pd.to_datetime(datetime.today().date())
limite = hoje - timedelta(days=180)

# Criando coluna status
def status_inspecao(row):
    if pd.isna(row['DATA_INSPECAO']):
        return 'Pendente (sem inspeção)'
    elif row['DATA_INSPECAO'] < limite:
        return 'Pendente (inspeção vencida)'
    else:
        return 'OK'

df['STATUS'] = df.apply(status_inspecao, axis=1)

# Filtros no dashboard
gerentes = ['Todos'] + sorted(df['GERENTE_IMEDIATO'].dropna().unique().tolist())
coordenadores = ['Todos'] + sorted(df['COORDENADOR_IMEDIATO'].dropna().unique().tolist())

gerente_selecionado = st.selectbox("Filtrar por Gerente:", gerentes)
coordenador_selecionado = st.selectbox("Filtrar por Coordenador:", coordenadores)

df_filtrado = df.copy()
if gerente_selecionado != 'Todos':
    df_filtrado = df_filtrado[df_filtrado['GERENTE_IMEDIATO'] == gerente_selecionado]
if coordenador_selecionado != 'Todos':
    df_filtrado = df_filtrado[df_filtrado['COORDENADOR_IMEDIATO'] == coordenador_selecionado]

# Agregação para gráfico
resumo = (
    df_filtrado.groupby(['GERENTE_IMEDIATO', 'COORDENADOR_IMEDIATO', 'STATUS'])
    .size()
    .reset_index(name='QTD')
)

if resumo.empty:
    st.warning("Nenhum dado para os filtros selecionados.")
else:
    fig = px.bar(
        resumo,
        x='COORDENADOR_IMEDIATO',
        y='QTD',
        color='STATUS',
        barmode='group',
        facet_col='GERENTE_IMEDIATO',
        title="Pendências e Inspeções OK por Coordenador e Gerente",
        labels={'COORDENADOR_IMEDIATO': 'Coordenador', 'QTD': 'Quantidade', 'STATUS': 'Status'}
    )
    st.plotly_chart(fig, use_container_width=True)

# Exibir tabela detalhada (opcional)
if st.checkbox("Mostrar dados detalhados"):
    st.dataframe(df_filtrado[['IDTEL_TECNICO', 'PRODUTO_PRINC_INSPECAO', 'DATA_INSPECAO', 'GERENTE_IMEDIATO', 'COORDENADOR_IMEDIATO', 'STATUS']])


2025-05-27 19:51:59.753 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 19:51:59.754 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 19:51:59.904 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-27 19:51:59.908 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 19:51:59.941 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 19:51:59.956 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-27 19:51:59.959 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [ ]:
!pip install streamlit plotly pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.1 MB/s eta 0:00:00
